In [16]:
import torch
import torchvision
import torch.nn.functional as F


# import cifar10 dataset, dataloader
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=torchvision.transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=2)

images = next(iter(train_loader))[0]

print(images.shape)

input_shape = images.shape

flatened = images.view(-1, 3)

# embddings copy of flatened
embeddings = flatened.detach().clone().T

similarity = torch.matmul(flatened, embeddings)
distances = (torch.sum(flatened ** 2, dim=1, keepdim=True)
                + torch.sum(embeddings ** 2, dim=0)
                - 2 * similarity)

encoding_indices = torch.argmin(distances, dim=1)

print(encoding_indices.shape)

encodings = F.one_hot(encoding_indices, num_classes=12288)
# encodings.shape = (batch_size * height * width, num_embeddings)

quantized = torch.matmul(encodings.float(), embeddings.t())
 # quantized.shape = (batch_size * height * width, embedding_dim)

quantized = quantized.view(input_shape) # Take this for the loss
# quantized.shape = (batch_size, embedding, height, width)

print(quantized.shape)
# plot restored and original with matplotlib

import matplotlib.pyplot as plt 

fig, ax = plt.subplots(1, 2)
ax[0].imshow(images[3].permute(1, 2, 0))
ax[1].imshow(quantized[3].permute(1, 2, 0))
plt.show()


print(flatened.shape)



torch.Size([12, 3, 32, 32])
torch.Size([12288])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (12288x65536 and 12288x3)